# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introducción

En la parte 1 de este desafío, se realizó el procesamiento previo y el entrenamiento de un modelo a partir de un conjunto de datos base proporcionados. En este segundo paso, se integrará todas las transformaciones y eventos de entrenamiento creados previamente en un Pipeline completo para *deploy* en **Watson Machine Learning**.

### Preparación del Notebook

Primero realizaremos la instalación do scikit-learn y la importación de las mismas bibliotecas utilizadas anteriormente

In [ ]:
!pip install scikit-learn==0.20.0 --upgrade

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

Es necesario volver a insertar el conjunto de datos base como un pandas dataframe, siguiendo las instrucciones

![alt text](https://i.imgur.com/K1DwL9I.png "importing-csv-as-df")

Después de seleccionar la opción **"Insert to code"**, la celda de abajo se llenará con el código necesario para importar y leer los datos en el archivo .csv como un Pandas DataFrame.

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6de7dd23c5d0484ab588e2ca41cb2faa = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vsu_vFJYT95osja2EPOX8ta8wDCUIcWLO6nbAGmsXb_P',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6de7dd23c5d0484ab588e2ca41cb2faa.get_object(Bucket='desafio2tortuga-donotdelete-pr-8vuqmljhaidmva',Key='dataset-tortuga-desafio-2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


Unnamed: 0               NAME   USER_ID  HOURS_DATASCIENCE  HOURS_BACKEND  \
0          28        Stormy Muto  58283940                7.0           39.0   
1          81       Carlos Ferro   1357218               32.0            0.0   
2          89  Robby Constantini  63212105               45.0            0.0   
3         138       Paul Mckenny  23239851               36.0           19.0   
4         143          Jean Webb  72234478               61.0           78.0   

   HOURS_FRONTEND  NUM_COURSES_BEGINNER_DATASCIENCE  \
0            29.0                               2.0   
1            44.0                               2.0   
2            59.0                               0.0   
3            28.0                               0.0   
4            38.0                               6.0   

   NUM_COURSES_BEGINNER_BACKEND  NUM_COURSES_BEGINNER_FRONTEND  \
0                           4.0                            0.0   
1                           0.0                            0.0   
2                           5.0                            4.0   
3                           5.0                            7.0   
4                          11.0                            0.0   

   NUM_COURSES_ADVANCED_DATASCIENCE  NUM_COURSES_ADVANCED_BACKEND  \
0                               2.0                           5.0   
1                               0.0                           5.0   
2                               0.0                           4.0   
3                               0.0                           5.0   
4                               4.0                           3.0   

   NUM_COURSES_ADVANCED_FRONTEND  AVG_SCORE_DATASCIENCE  AVG_SCORE_BACKEND  \
0                            0.0                   84.0               74.0   
1                            0.0                   67.0               45.0   
2                            1.0                    NaN               54.0   
3                            3.0                    NaN               71.0   
4                            0.0                   66.0               85.0   

   AVG_SCORE_FRONTEND                PROFILE  
0                 NaN     beginner_front_end  
1                 NaN     beginner_front_end  
2                47.0     advanced_front_end  
3                89.0  beginner_data_science  
4                 NaN     advanced_front_end

### Construcción del Pipeline completo para el encapsulamiento en WML

#### Preparando transformaciones personalizadas para cargar en WML

En el paso anterior, se mostró cómo crear una transformación personalizada, declarando una clase Python con los métodos ``fit`` y ``transform``.

    - Código de transformación personalizada DropColumns():
    
    from sklearn.base import BaseEstimator, TransformerMixin
    # All sklearn Transforms must have the `transform` and `fit` methods
    class DropColumns(BaseEstimator, TransformerMixin):
        def __init__(self, columns):
            self.columns = columns
        def fit(self, X, y=None):
            return self
        def transform(self, X):
            # Primero copiamos el dataframe de entrada 'X' de entrada
            data = X.copy()
            # Devolvemos un nuevo marco de datos sin las columnas no deseadas
            return data.drop(labels=self.columns, axis='columns')

Para integrar estos tipos de transformaciones personalizadas con Pipelines en Watson Machine Learning, primero debe empaquetar su código personalizado como una biblioteca de Python. Esto se puede hacer fácilmente usando la herramienta *setuptools*.

En el siguiente repositorio de git: https://github.com/vnderlev/sklearn_transforms tenemos todos los archivos necesarios para crear un paquete de Python, llamado **my_custom_sklearn_transforms**.
Este paquete tiene la siguiente estructura de archivos:

    /my_custom_sklearn_transforms.egg-info
        dependency_links.txt
        not-zip-safe
        PKG-INFO
        SOURCES.txt
        top_level.txt
    /my_custom_sklearn_transforms
        __init__.py
        sklearn_transformers.py
    PKG-INFO
    README.md
    setup.cfg
    setup.py
    
El archivo principal, que contendrá el código para nuestras transformaciones personalizadas, es el archivo **/my_custom_sklearn_transforms/sklearn_transformers.py**. Si accedes a él en el repositorio, notarás que contiene exactamente el mismo código declarado en el primer paso (la clase DropColumns).

Si has declarado sus propias transformaciones (además de la DropColumn proporcionada), debes agregar todas las clases de esas transformaciones creadas en este mismo archivo. Para hacer esto, debes hacer fork de este repositorio (esto se puede hacer en la propia interfaz web de Github, haciendo clic en el botón como se muestra en la imagen a continuación) y agregue sus clases personalizadas al archivo **sklearn_transformers.py**.

![alt text](https://i.imgur.com/2lZ4Ty2.png "forking-a-repo")

Si solo hizo uso de la transformación proporcionada (DropColumns), puede omitir este paso de fork y continuar usando el paquete base provisto. :)

Después de preparar su paquete de Python con sus transformaciones personalizadas, reemplace el enlace del repositorio de git en la celda a continuación y ejecútelo. Si no ha preparado ninguna transformación nueva, ejecute la celda con el enlace del repositorio ya proporcionado.

<hr>
    
**OBSERVACIÓN**

Si la ejecución de la celda a continuación devuelve un error de que el repositorio ya existe, ejecute:

**!rm -r -f sklearn_transforms**

In [3]:
# reemplace el enlace a continuación con el enlace de su repositorio de git (si corresponde)
!git clone https://github.com/vnderlev/sklearn_transforms.git

fatal: destination path 'sklearn_transforms' already exists and is not an empty directory.


In [4]:
!cd sklearn_transforms
!ls -ltr

total 68
drwxr-x--- 5 dsxuser dsxuser  4096 Aug 28 19:49 sklearn_transforms
-rw-r----- 1 dsxuser dsxuser 62140 Aug 28 19:49 sklearn_transforms.zip


Para cargar el código en WML, necesitamos enviar un archivo .zip con todo el código fuente, luego comprimiremos el directorio clonado a continuación:

In [5]:
!zip -r sklearn_transforms.zip sklearn_transforms

updating: sklearn_transforms/ (stored 0%)
updating: sklearn_transforms/.git/ (stored 0%)
updating: sklearn_transforms/.git/description (deflated 14%)
updating: sklearn_transforms/.git/objects/ (stored 0%)
updating: sklearn_transforms/.git/objects/82/ (stored 0%)
updating: sklearn_transforms/.git/objects/82/e0b0b52c39cd1840e7853e6d00986fae5683f4 (stored 0%)
updating: sklearn_transforms/.git/objects/2f/ (stored 0%)
updating: sklearn_transforms/.git/objects/2f/ab3357d7d157a7a6bbda08aad6cf19970d2cc3 (stored 0%)
updating: sklearn_transforms/.git/objects/5a/ (stored 0%)
updating: sklearn_transforms/.git/objects/5a/b47e5cada9bcefbd66782d50edf2a86005dcb5 (stored 0%)
updating: sklearn_transforms/.git/objects/da/ (stored 0%)
updating: sklearn_transforms/.git/objects/da/64b79708ea475e8c6b484d837676ce91d9801e (stored 0%)
updating: sklearn_transforms/.git/objects/af/ (stored 0%)
updating: sklearn_transforms/.git/objects/af/1140cfcb4e33bc03d9b067c1f7d70147d043d7 (stored 0%)
updating: sklearn_transfo

Con el archivo zip de nuestro paquete cargado en el Kernel de este notebook, podemos usar la herramienta pip para instalarlo, de acuerdo con la celda a continuación:

In [6]:
!pip install sklearn_transforms.zip

Processing ./sklearn_transforms.zip
  Stored in directory: /home/dsxuser/.tmp/pip-ephem-wheel-cache-i5434ey8/wheels/8f/88/32/f886e7510a37b111e2a1b7e689e04450acda46732970a7ed78
Successfully built my-custom-sklearn-transforms
  Found existing installation: my-custom-sklearn-transforms 1.0
    Uninstalling my-custom-sklearn-transforms-1.0:
      Successfully uninstalled my-custom-sklearn-transforms-1.0


¡Ahora podemos importar nuestro paquete personalizado a nuestro notebook!

Importaremos la transformación DropColumns. Si tienes otras transformaciones personalizadas, ¡no olvides importarlas!

In [7]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns

#### Declarando un Pipeline

Después de importar transformaciones personalizadas como un paquete de Python, podemos proceder a la declaración de nuestro Pipeline.

El proceso es muy similar al realizado en la primera etapa, pero con algunas diferencias importantes, ¡así que presta mucha atención!

El Pipeline de ejemplo tiene tres etapas: 

    - Remover las colunas "NAME" e "Unnamed: 0"
    - Asignar "ceros" a todos los valores faltantes
    - insertar datos preprocesados como entrada en un modelo entrenado
    
Recordando, la entrada de este Pipeline será el conjunto de datos brutos proporcionados, excepto la columna "PROFILE" (variable de objetivo que será determinada por el modelo).

Entonces tendremos 16 valores de entrada en el **PIPELINE** (en el modelo habrá 14 entradas, ya que las columnas "NAME" y "Unnamed: 0" se eliminarán en la primera etapa después de la transformación DropColumns).


    Unnamed: 0                          - Esta columna no tiene nombre y debe ser eliminada del dataset
    NAME                                - Nombre del estudiante
    USER_ID                             - Número de identificación del estudiante
    HOURS_DATASCIENCE                   - Número de horas de estudio en Data Science
    HOURS_BACKEND                       - Número de horas de estudio en Web (Back-End)
    HOURS_FRONTEND                      - Número de horas de estudio en Web (Front-End)
    NUM_COURSES_BEGINNER_DATASCIENCE    - Número de cursos de nivel principiante en Data Science completados por el estudiante
    NUM_COURSES_BEGINNER_BACKEND        - Número de cursos de nivel principiante en Web (Back-End) completados por el estudiante
    NUM_COURSES_BEGINNER_FRONTEND       - Número de cursos de nivel principiante en Web (Front-End) completados por el estudiante
    NUM_COURSES_ADVANCED_DATASCIENCE    - Número de cursos de nivel avanzado en Data Science completados por el estudiante
    NUM_COURSES_ADVANCED_BACKEND        - Número de cursos de nivel avanzado en Web (Back-End) completados por el estudiante
    NUM_COURSES_ADVANCED_FRONTEND       - Número de cursos de nivel avanzado en Web (Front-End) completados por el estudiante
    AVG_SCORE_DATASCIENCE               - Promedio acumulado en cursos de Data Science completados por el estudiante
    AVG_SCORE_BACKEND                   - Promedio acumulado en cursos de Web (Back-End) completados por el estudiante
    AVG_SCORE_FRONTEND                  - Promedio acumulado en cursos de Web (Front-End) completados por el estudiante

La salida del Pipeline será un valor estimado para la columna "PROFILE".

**¡¡ATENCIÓN!!**

La celda de arriba, aunque muy similar a la definición de características en la primera etapa de este desafío, ¡tiene una gran diferencia!

Contiene las columnas "NAME" y "Unnamed: 0" como features. Esto se debe a que en este caso estas son las entradas *PIPELINE*, no el modelo.

En la celda a continuación, se declara un objeto scikit-learn **Pipeline**, donde se declara el parámetro *steps*, que no es más que una lista de los pasos en nuestro pipeline:

    'remove_cols'     - transformación personalizada DropColumns
    'imputer'         - transformación scikit-learn incorporada para asignar valores faltantes
    'dtc'             - un clasificador a través del árbol de decisión

Tenga en cuenta que pasamos como transformaciones instanciadas anteriormente, bajo el nombre `rm_columns` y` si`.

Luego, ejecutaremos el método `fit ()` de Pipeline, realizando el preprocesamiento y entrenamiento del modelo a la vez.

### Crear Pipe Line

In [8]:
df_base=df_data_1.drop(columns=["USER_ID","NAME", "Unnamed: 0"], axis=1)        
df_base=df_data_1
datos_numericos = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_var_numericas = df_base.select_dtypes(include=datos_numericos).copy()
col_total_nulos = df_var_numericas.isnull().sum()
serie_col_nombres = col_total_nulos[col_total_nulos > 0]
for columna in serie_col_nombres.keys():
    mean_col=np.mean(df_var_numericas[columna])
    df_var_numericas[columna].replace(np.nan, mean_col, inplace=True)   
df_var_no_numericas = df_base[df_base.columns.difference(df_var_numericas.columns)].copy()
df_data = pd.concat([df_var_numericas,df_var_no_numericas],axis=1)
features = [
    "Unnamed: 0", "NAME", "USER_ID", "HOURS_DATASCIENCE", "HOURS_BACKEND", "HOURS_FRONTEND",
    "NUM_COURSES_BEGINNER_DATASCIENCE", "NUM_COURSES_BEGINNER_BACKEND", "NUM_COURSES_BEGINNER_FRONTEND",
    "NUM_COURSES_ADVANCED_DATASCIENCE", "NUM_COURSES_ADVANCED_BACKEND", "NUM_COURSES_ADVANCED_FRONTEND",
    "AVG_SCORE_DATASCIENCE", "AVG_SCORE_BACKEND", "AVG_SCORE_FRONTEND"
]
dfX=df_data[features]
y = df_data[['PROFILE']]
X_train, X_test, y_train, y_test = train_test_split(dfX, y,random_state=0)

rm_columns = DropColumns(
    columns=["Unnamed: 0", "NAME", "USER_ID"] 
)
    
pipe = Pipeline(steps=[('remove', rm_columns),('scaler', StandardScaler()), ('svc', SVC())])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9574

### Pruebas Pipe Line

In [10]:
data = np.array([[521,"Lance Rochester",19728116,49.0,44.0,85.0,2.0,3.0,5.0,6.0,4.0,2.0,45.0,89.0,61.0]])
# Creating pandas dataframe from numpy array
#521,Lance Rochester,19728116,49.0,44.0,85.0,2.0,3.0,5.0,6.0,4.0,2.0,45.0,89.0,61.0,beginner_data_science
df_test = pd.DataFrame(data,columns=features)
display(df_test)
pipe.predict(df_test)

Unnamed: 0             NAME   USER_ID HOURS_DATASCIENCE HOURS_BACKEND  \
0        521  Lance Rochester  19728116              49.0          44.0   

  HOURS_FRONTEND NUM_COURSES_BEGINNER_DATASCIENCE  \
0           85.0                              2.0   

  NUM_COURSES_BEGINNER_BACKEND NUM_COURSES_BEGINNER_FRONTEND  \
0                          3.0                           5.0   

  NUM_COURSES_ADVANCED_DATASCIENCE NUM_COURSES_ADVANCED_BACKEND  \
0                              6.0                          4.0   

  NUM_COURSES_ADVANCED_FRONTEND AVG_SCORE_DATASCIENCE AVG_SCORE_BACKEND  \
0                           2.0                  45.0              89.0   

  AVG_SCORE_FRONTEND  
0               61.0

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


array(['beginner_data_science'], dtype=object)

Ahora que tenemos un pipeline completo, con los pasos de preprocesamiento configurados y también un modelo por árbol de decisiones ya entrenado, ¡podemos integrarnos con Watson Machine Learning!

### Encapsulación de un Pipeline personalizado en Watson Machine Learning 

#### Establecer una conexión entre el cliente WML Python y su instancia de servicio en la nube

In [11]:
# Biblioteca de Python con implementación de un cliente HTTP para la API de WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Las siguientes celdas desplegarán el pipeline declarado en este notebook en WML. Continúe solo si ya está satisfecho con su modelo y cree que es hora de implementar su solución.

Pegue las credenciales de su instancia de Watson Machine Learning en la variable de la celda siguiente.

Es importante que la variable que contiene los valores tenga el nombre de `` wml_credentials`` para que las siguientes celdas de este notebook se ejecuten correctamente.

In [12]:
wml_credentials = {
  "apikey": "LxfAbLYf68dEjPXgKHOogY56Fjm7GpMtkN9ozH1pJcZN",
  "iam_apikey_description": "Auto-generated for key a41eca3c-4d08-4e15-b2d8-91cf57a8618f",
  "iam_apikey_name": "Credenciales de servicio-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/08f0243a8c1c4400816f658b23ae1bd1::serviceid:ServiceId-23fb29bc-3b7c-4852-a091-e335c9ab7669",
  "instance_id": "1b22fcab-92b0-4471-bd88-99c1498e21d2",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [13]:
# Creación de un objeto cliente de Watson Machine Learning a partir de las credenciales proporcionadas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

### ATENCIÓN, SI EJECUTA LAS 2 CELDAS SIGUIENTES BORRARÁ LOS DEPLIEGUES ANTERIORES DEL MODELO

In [14]:
#Celda para borrar los Deployments:

for uid in clientWML.deployments.get_uids():
    if clientWML.deployments.get_details(uid)['entity']['name'] == 'desafio-2-mbtc2020-deployment-es-2' :
        print('Deleted ' + clientWML.deployments.get_details(uid)['entity']['name'] )
        clientWML.deployments.delete(uid)

Deleted desafio-2-mbtc2020-deployment-es-2


In [15]:
#Celda para borrar todos los recursos:

d = clientWML.repository.get_details()

for k in d:
    for res in d[k]["resources"]:
        if res['entity']['name'] in ['my_custom_wml_runtime_es_2', 'my_custom_sklearn_transform_es_2', 'desafio-2-mbtc2020-pipeline-es-2']:
            clientWML.repository.delete(res["metadata"]["guid"])
            print('Deleted ' + res['entity']['name'])

Deleted desafio-2-mbtc2020-pipeline-es-2
Deleted my_custom_wml_runtime_es_2
Deleted my_custom_sklearn_transform_es_2


In [16]:
# Extracción de detalles de su instancia de Watson Machine Learning

instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

{
    "entity": {
        "source": "Bluemix",
        "published_models": {
            "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/1b22fcab-92b0-4471-bd88-99c1498e21d2/published_models"
        },
        "usage": {
            "capacity_units": {
                "current": 0,
                "limit": 180000000
            },
            "computation_time": {
                "current": 0,
                "limit": 180000
            },
            "deployment_count": {
                "current": 0,
                "limit": 5
            },
            "expiration_date": "2020-09-01T00:00:00.000Z",
            "gpu_count_k80": {
                "current": 0,
                "limit": 8
            },
            "gpu_count_p100": {
                "current": 0,
                "limit": 0
            },
            "gpu_count_v100": {
                "current": 0,
                "limit": 0
            },
            "model_count": {
                "current": 0,
         

**¡¡ATENCIÓN!!**

¡Preste atención a los límites de consumo de su instancia de Watson Machine Learning!

Si expira la capa libre, no podrá evaluar su modelo (¡ya que es necesario realizar algunas llamadas a API que consumen predicciones!)

#### Listado de todos los artefactos almacenados en su WML

Para listar todos los artefactos almacenados en Watson Machine Learning, puede utilizar la siguiente función:

    clientWML.repository.list()

In [17]:
# Listado de todos los artefactos almacenados en su WML

clientWML.repository.list()

----  ----  -------  ---------  ----
GUID  NAME  CREATED  FRAMEWORK  TYPE
----  ----  -------  ---------  ----


En el plan LITE de Watson Machine Learning, solo se puede implementar un modelo a la vez. Si ya tiene un modelo en línea en su instancia, puede eliminarlo usando el método clientWML.repository.delete ():

    artifact_guid = "359c8951-d2fe-4063-8706-cc06b32d5e0d"
    clientWML.repository.delete(artifact_guid)

#### Crear una nueva definición de paquete Python personalizada en WML

El primer paso para realizar su implementación es almacenar el código de las transformaciones personalizadas creadas.

Para este paso solo necesitamos el archivo .zip del paquete creado (¡que ya hemos cargado en el Kernel!)

In [18]:
# Definición de metadatos de nuestro paquete con transformaciones personalizadas 
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_es_2",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip creado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

print("\n Lista de artefactos en tiempo de ejecución almacenados en WML:")
clientWML.repository.list()


 Lista de artefactos en tiempo de ejecución almacenados en WML:
------------------------------------  --------------------------------  ------------------------  ---------  --------------
GUID                                  NAME                              CREATED                   FRAMEWORK  TYPE
ae0e078b-c4d9-4e72-84bb-e75d1199ad23  my_custom_sklearn_transform_es_2  2020-08-28T20:25:06.580Z  -          python library
------------------------------------  --------------------------------  ------------------------  ---------  --------------


#### Creación de una nueva definición personalizada de runtime Python en WML

El segundo paso es almacenar una definición de runtime Python para usar nuestra biblioteca personalizada.

Esto puede hacerse de la siguiente manera:

In [19]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_es_2",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalles de runtime almacenados:")
print(json.dumps(runtime_details, indent=4))


 Detalles de runtime almacenados:
{
    "entity": {
        "name": "my_custom_wml_runtime_es_2",
        "description": "A Python runtime with custom sklearn Transforms",
        "custom_libraries": [
            {
                "name": "my_custom_sklearn_transform_es_2",
                "url": "https://private.us-south.ml.cloud.ibm.com/v4/libraries/ae0e078b-c4d9-4e72-84bb-e75d1199ad23",
                "version": "1.0"
            }
        ],
        "content_url": "https://private.us-south.ml.cloud.ibm.com/v4/runtimes/da82a769-d760-47e2-ba29-ccaa7d524647/content",
        "platform": {
            "name": "python",
            "version": "3.6"
        }
    },
    "metadata": {
        "created_at": "2020-08-28T20:25:13.155Z",
        "guid": "da82a769-d760-47e2-ba29-ccaa7d524647",
        "url": "https://us-south.ml.cloud.ibm.com/v4/runtimes/da82a769-d760-47e2-ba29-ccaa7d524647"
    }
}


In [20]:
# Listando todos runtimes armazenados no seu WML:
clientWML.runtimes.list()

------------------------------------  --------------------------  ------------------------  ----------
GUID                                  NAME                        CREATED                   PLATFORM
da82a769-d760-47e2-ba29-ccaa7d524647  my_custom_wml_runtime_es_2  2020-08-28T20:25:13.155Z  python-3.6
------------------------------------  --------------------------  ------------------------  ----------


#### Crear una nueva definición de Pipeline personalizado en WML

Finalmente crearemos una definición (metadatos) para que nuestro Pipeline se aloje en WML.

Definimos como parámetros un nombre para el artefacto y el ID de runtime creado anteriormente.

In [21]:
model_meta = {    
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-es-2',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Luego llamamos al método para almacenar la nueva definición:

In [22]:
# Función para almacenar una definición de Pipeline en WML 
stored_model_details = clientWML.repository.store_model(
    model=pipe,  # `my_pipeline` es la variable creada previamente y contiene nuestro Pipeline ya entrenado :)
    meta_props=model_meta,  # Metadatos definidos en la celda anterior
    training_data=None  # No cambie este parámetro
)

print("\n Lista de artefactos almacenados en WML:")
clientWML.repository.list()

# Detalles del modelo alojado en Watson Machine Learning
print("\n Metadatos almacenados del modelo:")
print(json.dumps(stored_model_details, indent=4))


 Lista de artefactos almacenados en WML:
------------------------------------  --------------------------------  ------------------------  -----------------  --------------
GUID                                  NAME                              CREATED                   FRAMEWORK          TYPE
287513b2-b45e-42f8-b1ce-43a354214186  desafio-2-mbtc2020-pipeline-es-2  2020-08-28T20:25:23.626Z  scikit-learn-0.20  model
ae0e078b-c4d9-4e72-84bb-e75d1199ad23  my_custom_sklearn_transform_es_2  2020-08-28T20:25:06.580Z  -                  python library
da82a769-d760-47e2-ba29-ccaa7d524647  my_custom_wml_runtime_es_2        2020-08-28T20:25:13.155Z  -                  python runtime
------------------------------------  --------------------------------  ------------------------  -----------------  --------------

 Metadatos almacenados del modelo:
{
    "metadata": {
        "guid": "287513b2-b45e-42f8-b1ce-43a354214186",
        "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/1b22fc

#### Despliegue de su modelo para consumo inmediato por otras aplicaciones

In [23]:
# El modelo finalmente se implementa usando el método `` deployments.create () ``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # No cambie este parámetro
    name="desafio-2-mbtc2020-deployment-es-2",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # No cambie este parámetro
    deployment_type='online',  # No cambie este parámetro
    deployment_format='Core ML',  # No cambie este parámetro
    meta_props=model_meta  # No cambie este parámetro
)



#######################################################################################

Synchronous deployment creation for uid: '287513b2-b45e-42f8-b1ce-43a354214186' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS..
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='626baf43-a3d4-4339-8341-17f012be3b24'
------------------------------------------------------------------------------------------------




#### Prueba de un modelo alojado en Watson Machine Learning

In [24]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("Su URL de llamada a la API es: {}".format(model_endpoint_url))

Su URL de llamada a la API es: https://us-south.ml.cloud.ibm.com/v3/wml_instances/1b22fcab-92b0-4471-bd88-99c1498e21d2/deployments/626baf43-a3d4-4339-8341-17f012be3b24/online


In [25]:
# Detalles del despliegue realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # este es su ID de implementación!
)

print("Metadatos de despliegue realizado: \n")
print(json.dumps(deployment_details, indent=4))

Metadatos de despliegue realizado: 

{
    "metadata": {
        "guid": "626baf43-a3d4-4339-8341-17f012be3b24",
        "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/1b22fcab-92b0-4471-bd88-99c1498e21d2/deployments/626baf43-a3d4-4339-8341-17f012be3b24",
        "created_at": "2020-08-28T20:25:33.669Z",
        "modified_at": "2020-08-28T20:25:34.995Z"
    },
    "entity": {
        "runtime_environment": "python-3.6",
        "name": "desafio-2-mbtc2020-deployment-es-2",
        "scoring_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/1b22fcab-92b0-4471-bd88-99c1498e21d2/deployments/626baf43-a3d4-4339-8341-17f012be3b24/online",
        "deployable_asset": {
            "name": "desafio-2-mbtc2020-pipeline-es-2",
            "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/1b22fcab-92b0-4471-bd88-99c1498e21d2/published_models/287513b2-b45e-42f8-b1ce-43a354214186",
            "guid": "287513b2-b45e-42f8-b1ce-43a354214186",
            "description": "my

In [35]:
scoring_payload = {
    'fields': [
        "Unnamed: 0", "NAME", "USER_ID","HOURS_DATASCIENCE", "HOURS_BACKEND", "HOURS_FRONTEND",
        "NUM_COURSES_BEGINNER_DATASCIENCE", "NUM_COURSES_BEGINNER_BACKEND", "NUM_COURSES_BEGINNER_FRONTEND",
        "NUM_COURSES_ADVANCED_DATASCIENCE", "NUM_COURSES_ADVANCED_BACKEND", "NUM_COURSES_ADVANCED_FRONTEND",
        "AVG_SCORE_DATASCIENCE", "AVG_SCORE_BACKEND", "AVG_SCORE_FRONTEND"
    ],
    'values': [
        [
            2868,"Charles Travillian",21668235,33.0,42.0,59.0,2.0,1.0,3.0,6.0,2.0,7.0,58.0,92.0,87.0,
        ]
    ]
}
#0,"Paula Waters",85123728,
#89,Robby Constantini,63212105,45.0,0.0,59.0,0.0,5.0,4.0,0.0,4.0,1.0,,54.0,47.0
# Definición de la variable objetivo
#517,Ursula Kuehnert,34472788,68.0,54.0,29.0,4.0,8.0,5.0,6.0,4.0,3.0,64.0,69.0,50.0,advanced_front_end
#518,John Shear,41667523,18.0,48.0,10.0,2.0,4.0,5.0,3.0,7.0,0.0,82.0,75.0,57.0,beginner_backend
#576,Patricia Jones,47901337,40.0,39.0,63.0,4.0,3.0,6.0,0.0,1.0,1.0,65.0,30.0,54.0,advanced_data_science
#644,Carmen Clanton,52710878,0.0,92.0,57.0,5.0,3.0,4.0,4.0,5.0,5.0,74.0,74.0,67.0,advanced_front_end
#645,Charles Swindle,01539545,32.0,69.0,47.0,2.0,5.0,4.0,6.0,5.0,2.0,80.0,58.0,79.0,advanced_front_end
#646,Sharon Carter,63214368,30.0,74.0,40.0,5.0,5.0,7.0,2.0,5.0,2.0,54.0,81.0,63.0,advanced_data_science
#647,Jasper Yan,63875515,0.0,91.0,77.0,4.0,1.0,5.0,5.0,8.0,7.0,63.0,55.0,66.0,advanced_front_end
#648,Barbara Langham,51438625,2.0,58.0,21.0,5.0,6.0,8.0,6.0,1.0,10.0,68.0,78.0,54.0,advanced_backend
#649,William Garcia,60371163,20.0,14.0,41.0,1.0,4.0,9.0,4.0,5.0,2.0,75.0,89.0,74.0,beginner_data_science
#650,William Jarvis,06467007,68.0,58.0,16.0,5.0,4.0,6.0,6.0,4.0,6.0,61.0,60.0,47.0,advanced_backend
#651,Edith Wright,52060012,12.0,101.0,62.0,5.0,5.0,2.0,6.0,7.0,4.0,72.0,84.0,74.0,advanced_front_end
#2868,Charles Travillian,21668235,33.0,42.0,59.0,2.0,1.0,3.0,6.0,2.0,7.0,58.0,92.0,87.0,beginner_backend
#2869,Jung Garcia,77826164,76.0,11.0,37.0,5.0,4.0,2.0,6.0,4.0,4.0,46.0,73.0,79.0,advanced_data_science
#2870,Bobbie Cervone,44356814,78.0,33.0,48.0,4.0,7.0,4.0,6.0,1.0,2.0,50.0,62.0,59.0,beginner_front_end
#2871,Stacey Getz,23217194,12.0,60.0,32.0,3.0,8.0,5.0,6.0,4.0,4.0,87.0,78.0,63.0,advanced_front_end


target = ["PROFILE"]

print("\n Payload de datos para clasificar:")
print(json.dumps(scoring_payload, indent=4))


 Payload de datos para clasificar:
{
    "fields": [
        "Unnamed: 0",
        "NAME",
        "USER_ID",
        "HOURS_DATASCIENCE",
        "HOURS_BACKEND",
        "HOURS_FRONTEND",
        "NUM_COURSES_BEGINNER_DATASCIENCE",
        "NUM_COURSES_BEGINNER_BACKEND",
        "NUM_COURSES_BEGINNER_FRONTEND",
        "NUM_COURSES_ADVANCED_DATASCIENCE",
        "NUM_COURSES_ADVANCED_BACKEND",
        "NUM_COURSES_ADVANCED_FRONTEND",
        "AVG_SCORE_DATASCIENCE",
        "AVG_SCORE_BACKEND",
        "AVG_SCORE_FRONTEND"
    ],
    "values": [
        [
            2868,
            "Charles Travillian",
            21668235,
            33.0,
            42.0,
            59.0,
            2.0,
            1.0,
            3.0,
            6.0,
            2.0,
            7.0,
            58.0,
            92.0,
            87.0
        ]
    ]
}


In [36]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))


 Resultados:
{
    "fields": [
        "prediction"
    ],
    "values": [
        [
            "beginner_backend"
        ]
    ]
}


<hr>

## ¡Felicidades! 

Si todo salió bien, ¡ya tiene un clasificador basado en aprendizaje automático encapsulado como una API REST!

Para probar tu solución integrada con un asistente virtual y realizar el envío, visita la página:

https://tortuga.maratona.dev

Necesitarás la URL del endpoint de tu modelo y las credenciales WML :)